#### Run the following script to create a huge dataset containing the following: All images with objects from all datasets but norway. ca. 10% images without objects compared to those without per dataset. For Norway we crop the bottom 2880 x 1920 of the original images that contain objects, further split them into  6 960 x 960 images with their own .xml files containing their bounding boxes, and then save all images containg bounding boxes and only 10% of those not containing bounding boxes. Further, we choose about 300 of the original images that does not contain any bounding boxes and add them aswell.

In [66]:
import os
import random
import numpy as np
from xml.etree import ElementTree
from xml.dom import minidom
from PIL import Image
import json
import numpy as np
import shutil
import xml.etree.ElementTree as ET
%matplotlib inline

np.random.seed(42)

In [67]:
cwd = os.getcwd()
base_path = cwd
data_folder_name = 'RDD2022'
data_folder_path = os.path.join(cwd, data_folder_name)
dir_name = data_folder_path
folder_names = ['China_Drone', 'China_MotorBike', 'Czech', 'India', 'Japan', 'Norway', 'United_States']


In [68]:
with open('image_sizes.json') as json_file:
    sizes_dict = json.load(json_file)

In [69]:
def get_bounding_box_size_and_missing_files(folder_name):
    annotations_folder = os.path.join(data_folder_path, folder_name, 'train', 'annotations', 'xmls')
    box_sizes = []
    missing_files = []
    for file in os.listdir(annotations_folder):
        if file.endswith('.xml'):
            xml_path = os.path.join(annotations_folder, file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            has_object = False
            for obj in root.findall('object'):
                obj_class = obj.find('name').text
                if obj_class in ['D00', 'D10', 'D20', 'D40']:
                    has_object = True
                    bbox = obj.find('bndbox')
                    xmin = float(bbox.find('xmin').text)
                    ymin = float(bbox.find('ymin').text)
                    xmax = float(bbox.find('xmax').text)
                    ymax = float(bbox.find('ymax').text)
                    box_width = xmax - xmin
                    box_height = ymax - ymin
                    box_sizes.append((box_width, box_height))
            if not has_object:
                missing_files.append(file.replace('.xml', ''))
    return box_sizes, missing_files

In [70]:
dic = dict()
for folder_name in folder_names:
    dic[folder_name] = get_bounding_box_size_and_missing_files(folder_name)

contains_objects_dic = {}
for key, value in dic.items():
    contains_objects_dic[key] = [len(sizes_dict[key]), len(value[1]), len(sizes_dict[key]) - len(value[1])]

for key, value in dic.items():
    print('For dataset', key, 'we have:')
    print('Bound box statistics:')
    print(f'Num objects: {len(value[0])}, average box size = {np.mean(value[0])}, median box size = {np.median(value[0])}, min box size = {np.min(value[0])}, max box size = {np.max(value[0])}')
    print()

print(f'files without bounding boxes example: {dic["Norway"][1][0]}')

For dataset China_Drone we have:
Bound box statistics:
Num objects: 3068, average box size = 125.63820078226858, median box size = 81.5, min box size = 10.0, max box size = 511.0

For dataset China_MotorBike we have:
Bound box statistics:
Num objects: 4650, average box size = 156.9647311827957, median box size = 100.0, min box size = 4.0, max box size = 511.0

For dataset Czech we have:
Bound box statistics:
Num objects: 1745, average box size = 77.65730659025787, median box size = 65.0, min box size = 9.0, max box size = 550.0

For dataset India we have:
Bound box statistics:
Num objects: 6831, average box size = 144.3850095154443, median box size = 108.0, min box size = 12.0, max box size = 719.0

For dataset Japan we have:
Bound box statistics:
Num objects: 16470, average box size = 126.34043715846994, median box size = 92.0, min box size = 0.0, max box size = 1045.0

For dataset Norway we have:
Bound box statistics:
Num objects: 11229, average box size = 222.80177175171428, median 

In [71]:
number_of_objectless_images_to_include = {}
for key, value in contains_objects_dic.items():
    number_of_objectless_images_to_include[key] = int(value[2] * 0.1)

In [72]:
def generate_xml_file(filename, width, height, objects, save_path):
    # Create the root element and set the version and encoding attributes
    root = ET.Element('annotation')
    root.set('version', '1.0')
    root.set('encoding', 'UTF-8')

    # Create the folder, filename, size, and segmented elements and add them to the root
    folder = ET.SubElement(root, 'folder')
    folder.text = 'images'
    filename_elem = ET.SubElement(root, 'filename')
    filename_elem.text = filename
    size = ET.SubElement(root, 'size')
    width_elem = ET.SubElement(size, 'width')
    width_elem.text = str(width)
    height_elem = ET.SubElement(size, 'height')
    height_elem.text = str(height)
    depth = ET.SubElement(size, 'depth')
    segmented = ET.SubElement(root, 'segmented')
    segmented.text = '0'

    # Iterate over the objects and create an object element for each one
    for obj in objects:
        obj_elem = ET.SubElement(root, 'object')

        # Create the name, truncated, occluded, difficult, and bndbox elements and add them to the object element
        name = ET.SubElement(obj_elem, 'name')
        name.text = obj[0]
        truncated = ET.SubElement(obj_elem, 'truncated')
        truncated.text = '0'
        occluded = ET.SubElement(obj_elem, 'occluded')
        occluded.text = '0'
        difficult = ET.SubElement(obj_elem, 'difficult')
        difficult.text = '0'
        bndbox = ET.SubElement(obj_elem, 'bndbox')
        xmin = ET.SubElement(bndbox, 'xmin')
        xmin.text = str(obj[1])
        ymin = ET.SubElement(bndbox, 'ymin')
        ymin.text = str(obj[3])
        xmax = ET.SubElement(bndbox, 'xmax')
        xmax.text = str(obj[2])
        ymax = ET.SubElement(bndbox, 'ymax')
        ymax.text = str(obj[4])

        # Create the attributes element and add a rotation attribute to it
        attributes = ET.SubElement(obj_elem, 'attributes')
        attribute = ET.SubElement(attributes, 'attribute')
        name = ET.SubElement(attribute, 'name')
        name.text = 'rotation'
        value = ET.SubElement(attribute, 'value')
        value.text = '0.0'

    # Create the XML file and write it to the specified path
    tree = ET.ElementTree(root)
    tree.write(save_path)

In [73]:
ids = ['a', 'b', 'c', 'd', 'e', 'f']
from PIL import Image

def is_bbox_in_crop(xmin, xmax, ymin, ymax, left, top, right, bottom):
    return xmin >= left and xmax <= right and ymin >= top and ymax <= bottom

def is_bbox_outside_crop(xmin, xmax, ymin, ymax, left, top, right, bottom):
    return xmin >= right or xmax <= left or ymin >= bottom or ymax <= top

def crop_and_save_imgs_and_xmls_by_folder_name(folder_name, x_interval, y_interval, fraction_of_images_without_bb_to_save: float = 0.1):
    annotations_folder = os.path.join(data_folder_path, folder_name, 'train', 'annotations', 'xmls')
    image_folder = os.path.join(data_folder_path, folder_name, 'train', 'images')
    annotations_save_folder = os.path.join(data_folder_path.replace('RDD2022', 'RDD2022_modified'), folder_name, 'train', 'annotations', 'xmls')
    image_save_folder = os.path.join(data_folder_path.replace('RDD2022', 'RDD2022_modified'), folder_name, 'train', 'images')
    files_without_bb = dic[folder_name][1]
    for file in os.listdir(annotations_folder):
        if file.endswith('.xml'):
            if file in files_without_bb:
                continue
            filename = file.replace('.xml', '')
            xml_path = os.path.join(annotations_folder, file)
            tree = ET.parse(xml_path)
            root = tree.getroot()
            image_path = os.path.join(image_folder, file.replace('.xml', '.jpg'))
            with Image.open(image_path) as im:
                # Get the size of the input image
                img_width, img_height = im.size
                 # Create 3 x 2 grid of images
                 # Top should be a lower number than bottom
                left_pixels = [x_interval * i for i in range(3)]
                top_pixels = [img_height - y_interval * i for i in range(1, 3)]
                right_pixels = [x_interval * i for i in range(1, 4)]
                bottom_pixels = [img_height - y_interval * i for i in range(2)]
                # Adjust to account that pixel 0 in y directuon is at the top
                image_combos = []
                for indy, bottom in enumerate(bottom_pixels):
                    top = top_pixels[indy]
                    for indx, left in enumerate(left_pixels):
                        right = right_pixels[indx]
                    
                        image_combos.append((left, top, right, bottom))

                for indx, combo in enumerate(image_combos):
                    cropped_im = im.crop(combo)
                    bounding_boxes_to_save = []
                    for obj in root.findall('object'):
                        obj_class = obj.find('name').text
                        if obj_class in ['D00', 'D10', 'D20', 'D40']:
                            bbox = obj.find('bndbox')
                            xmin = float(bbox.find('xmin').text)
                            xmax = float(bbox.find('xmax').text)
                            ymin = float(bbox.find('ymin').text)
                            ymax = float(bbox.find('ymax').text)
                            if is_bbox_outside_crop(xmin, xmax, ymin, ymax, *combo):
                                continue
                            if is_bbox_in_crop(xmin, xmax, ymin, ymax, *combo):
                                pass
                            else:
                                # Bbox is partially in crop
                                # Calculate new coordinates
                                xmin = max(xmin, combo[0])
                                xmax = min(xmax, combo[2])
                                ymin = max(ymin, combo[1])
                                ymax = min(ymax, combo[3])
                            # Adjust coordinates to be relative to the crop
                            xmin = xmin - combo[0]
                            xmax = xmax - combo[0]
                            ymin = ymin - combo[1]
                            ymax = ymax - combo[1]
                            bounding_boxes_to_save.append((obj_class, xmin, xmax, ymin, ymax))
                    if len(bounding_boxes_to_save) > 0:
                        cropped_im.save(os.path.join(image_save_folder, filename + '_' + ids[indx] + '.jpg'))
                        xml_save_path = os.path.join(annotations_save_folder, filename + '_' + ids[indx] + '.xml')
                        generate_xml_file(filename + '_' + ids[indx] + '.jpg', x_interval, y_interval, bounding_boxes_to_save, xml_save_path)
                    else:
                        if random.random() < fraction_of_images_without_bb_to_save:
                            cropped_im.save(os.path.join(image_save_folder, filename + '_' + ids[indx] + '.jpg'))
                            xml_save_path = os.path.join(annotations_save_folder, filename + '_' + ids[indx] + '.xml')
                            generate_xml_file(filename + '_' + ids[indx] + '.jpg', x_interval, y_interval, bounding_boxes_to_save, xml_save_path)
                    


In [74]:
def create_folder_structure(root_directory):
    # Create the root directory
    os.makedirs(root_directory, exist_ok=True)
    
    # Define the subdirectory names and their structure
    subdirs = {
        "China_Drone": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "China_MotorBike": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "Czech": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "India": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "Japan": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "Norway": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        },
        "United_States": {
            "train": {
                "annotations": {
                    "xmls": None
                },
                "images": None
            }
        }
    }
    
    # Create the subdirectories and their structure
    for subdir, structure in subdirs.items():
        subdir_path = os.path.join(root_directory, subdir)
        os.makedirs(subdir_path, exist_ok=True)
        
        for subsubdir, substructure in structure.items():
            subsubdir_path = os.path.join(subdir_path, subsubdir)
            os.makedirs(subsubdir_path, exist_ok=True)
            
            for subsubsubdir, subsubstructure in substructure.items():
                subsubsubdir_path = os.path.join(subsubdir_path, subsubsubdir)
                os.makedirs(subsubsubdir_path, exist_ok=True)
                
                if subsubstructure:
                    for subsubsubsubdir, _ in subsubstructure.items():
                        subsubsubsubdir_path = os.path.join(subsubsubdir_path, subsubsubsubdir)
                        os.makedirs(subsubsubsubdir_path, exist_ok=True)
                
    print(f"Created folder structure in {root_directory}")
create_folder_structure('RDD2022_modified')

Created folder structure in RDD2022_modified


In [75]:
#chatGPT generated
def copy_images_and_annotations_without_bb(folder_name, num_images_to_copy):
    folder = folder_name
    image_path=os.path.join(data_folder_path, folder, 'train', 'images')
    annotation_path=os.path.join(data_folder_path, folder, 'train', 'annotations', 'xmls')
    image_save_folder=os.path.join('RDD2022_modified', folder, 'train', 'images')
    annotation_save_folder=os.path.join('RDD2022_modified', folder, 'train', 'annotations', 'xmls')

    all_files_names_without_objects = dic[folder_name][1]
    random.shuffle(all_files_names_without_objects)
    files_name = all_files_names_without_objects[:num_images_to_copy]
    for filename in files_name:
        img_path =os.path.join(image_path, filename + '.jpg')
        xml_path =os.path.join(annotation_path, filename + '.xml')
        img_save_path = os.path.join(image_save_folder, filename + '.jpg')
        xml_save_path = os.path.join(annotation_save_folder, filename + '.xml')
        shutil.copy(img_path, img_save_path)
        shutil.copy(xml_path, xml_save_path)


In [76]:
def copy_images_and_annotations_with_bb(folder_name):
    folder = folder_name
    image_path=os.path.join(data_folder_path, folder, 'train', 'images')
    annotation_path=os.path.join(data_folder_path, folder, 'train', 'annotations', 'xmls')
    image_save_folder=os.path.join('RDD2022_modified', folder, 'train', 'images')
    annotation_save_folder=os.path.join('RDD2022_modified', folder, 'train', 'annotations', 'xmls')

    all_files_names = os.listdir(image_path)
    all_files_names = [file_name.split('.')[0] for file_name in all_files_names]
    all_files_names_without_objects = dic[folder_name][1]
    all_files_names_with_objects = list(set(all_files_names) - set(all_files_names_without_objects))
    random.shuffle(all_files_names_with_objects)
    for filename in all_files_names_with_objects:
        img_path =os.path.join(image_path, filename + '.jpg')
        xml_path =os.path.join(annotation_path, filename + '.xml')
        img_save_path = os.path.join(image_save_folder, filename + '.jpg')
        xml_save_path = os.path.join(annotation_save_folder, filename + '.xml')
        shutil.copy(img_path, img_save_path)
        shutil.copy(xml_path, xml_save_path)

In [77]:
x_interval = 960
y_interval = 960
for folder in folder_names:
    if folder == 'Norway':
        crop_and_save_imgs_and_xmls_by_folder_name(folder, x_interval, y_interval)
    else:
        copy_images_and_annotations_with_bb(folder)
    copy_images_and_annotations_without_bb(folder, number_of_objectless_images_to_include[folder])


img_width, img_height: 
3650 2044
img_width, img_height: 
3643 2041
img_width, img_height: 
3643 2041
img_width, img_height: 
4040 2035
img_width, img_height: 
3643 2041
img_width, img_height: 
3650 2044
img_width, img_height: 
3643 2041
img_width, img_height: 
4040 2035
img_width, img_height: 
3650 2044
Occured for file: Norway_000008_a.jpg
Occured for file: Norway_000008_b.jpg
img_width, img_height: 
3650 2044
Occured for file: Norway_000009_a.jpg
Occured for file: Norway_000009_a.jpg
Occured for file: Norway_000009_b.jpg
Occured for file: Norway_000009_b.jpg
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
3643 2041
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
4040 2035
img_width, img_height: 
3650 2044
img_width, img_height: 
3650 2044
img_width, img_height: 


KeyboardInterrupt: 